In [80]:
f_path = 'C:\\Users\\Ali Salem\\Desktop\\App_Update\\static/files\\parameters.xlsx'
pricing_cbc_path = 'C:\\Users\\Ali Salem\\Desktop\\App_Update\\Pricing CBC/Pricing CBC Datasets Test\\Walmart CBC Extract incl BrandxSector Sourcing.xlsx'
slides_name = []

In [81]:

import os
%run "{os.path.dirname(os.getcwd())}\general_functions\generalFunctions.ipynb" #container

%run "{os.getcwd()}\Pricing CBC Replacement Function.ipynb" #container
%run "{os.path.dirname(os.getcwd())}\general_functions\Extracting Data Functions.ipynb"

In [82]:
from pptx import Presentation
import win32com.client as win32
import pandas as pd
import numpy as np
from pathlib import Path
import re
import sys 
win32c = win32.constants
import shutil
import os
from win32com import client
from pptx.util import Pt
from pptx.enum.text import PP_ALIGN
from pptx.chart.data import CategoryChartData,XyChartData,BubbleChartData
import win32com.client
from pptx.dml.color import RGBColor
from pptx.util import Inches,Cm
from pptx.chart.data import ChartData
from pptx.enum.chart import XL_TICK_LABEL_POSITION
from pptx.enum.chart import XL_LABEL_POSITION
from win32com.client import constants as xl
from pptx.enum.chart import XL_CHART_TYPE
import pickle
from pptx.enum.dml import MSO_LINE,MSO_LINE_DASH_STYLE
import time
import itertools
from pptx.enum.chart import XL_AXIS_CROSSES,XL_LEGEND_POSITION
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict 
from io import BytesIO 
import math
from pptx.enum.shapes import MSO_SHAPE
from pptx.oxml.xmlchemy import OxmlElement
from pptx.enum.text import PP_ALIGN, MSO_ANCHOR
from pptx.oxml.ns import qn
import adodbapi
from concurrent.futures import ThreadPoolExecutor, as_completed

In [83]:
filename = 'parameters.xlsx'

# Get the current working directory
current_dir = os.path.dirname(os.getcwd())

# Construct the full path to the file
f_path = os.path.join(current_dir, filename)
print(f_path)
#xls = pd.ExcelFile(f_path)
parm = pd.read_excel(f_path, sheet_name='Pricing_CBC')
fields = dict(zip(parm['Field'],parm['Value']))

c:\Users\Ali Salem\Desktop\App_Update\parameters.xlsx


In [84]:
server = "powerbi://api.powerbi.com/v1.0/myorg/"+ fields['server']
dataset_name = fields['dataset_name']
f_name = fields['f_name']
directory = r"Pricing CBC Datasets Test"
f_name = os.path.join(directory, f_name) 
file_path = os.path.join(os.getcwd(), f_name)

market =  list(set(fields['market'].split(','))-set(['']))
channel =  list(set(fields['channel'].split(','))-set(['']))
region = list(set(fields['region'].split(','))-set(['']))
client_manuf = list(set(fields['client_manuf'].split(','))-set(['']))
categories = list(set(fields['categories'].split(','))-set(['']))

pricingPlus = fields['pricingPlus'].replace("'", "")
pricingMinus = fields['pricingMinus'].replace("'", "")

currency = fields['currency']
decimals = fields['decimals']
sign = fields['sign']
data_source = fields['data_source']
marketsub= " ".join(market+channel+region)

end_date= fields['end_date']
past_12_months = pd.date_range(end=end_date, periods=12, freq='ME').strftime('%b-%y').tolist()
p12m_dax = "{" + ", ".join(f'"{date}"' for date in past_12_months) + "}"

entity_hierarchy = [
    ("Region", region),
    ("Channel", channel),
    ("Market", market)
]
hierarchy_levels = [
    ("Category", categories),
]

colorList = [
    RGBColor(0, 160, 151),
    RGBColor(126, 202, 196),
    RGBColor(0, 108, 109),
    RGBColor(146, 208, 80),
    RGBColor(0, 176, 80),
    RGBColor(184, 182, 13),
    RGBColor(0, 142, 135),   # New color close to the first and third colors
    RGBColor(131, 218, 212)  # New color close to the second color
]

<!-- ### Parameters -->

In [85]:
BrandelRun= fields['BrandelRun']
BrandSourcingRun= fields['BrandSourcingRun']
ProductSourcingRun= fields['ProductSourcingRun']
PERun= fields['PERun']
RevenueResponseRun= fields['RevenueResponseRun']
SERun= fields['SERun']

In [86]:
# file_path = r"C:\Users\Salma Hany\Documents\Slide-Automate\Pricing CBC Slide Duplicate\Pricing CBC Datasets\Edgewell Mexico CBC Extract.xlsx"

# market = []
# channel=[]
# region=['Walmart']
# marketsub=" ".join(market+channel+region)
# client_manuf = ["Edgewell Personal Care", "PBG"]
# categories = ["Manual Shave Men"]

# pricingPlus = "+15%"
# pricingMinus = "-10%"

# colorList = [
#     RGBColor(0, 160, 151),
#     RGBColor(126, 202, 196),
#     RGBColor(0, 108, 109),
#     RGBColor(146, 208, 80),
#     RGBColor(0, 176, 80),
#     RGBColor(184, 182, 13),
#     RGBColor(0, 142, 135),   # New color close to the first and third colors
#     RGBColor(131, 218, 212)  # New color close to the second color
# ]
# currency = '$' 
# sign = "before"
# decimals = 2
# data_source = "DATA SOURCE: Consumer Test | July 2025"

# server = "powerbi://api.powerbi.com/v1.0/myorg/Edgewell"
# dataset_name = "Edgewell US Male Dataset"
# entity_hierarchy = [
#     ("Region", region),
#     ("Channel", channel),
#     ("Market", market)
# ]
# hierarchy_levels = [
#     ("Category", categories),
# ]

# path=os.path.join(os.getcwd(),"Pricing CBC Datasets")

# Format months for DAX
# end_date = "2025-05-01"
# past_12_months = pd.date_range(end=end_date, periods=12, freq='ME').strftime('%b-%y').tolist()
# p12m_dax = "{" + ", ".join(f'"{date}"' for date in past_12_months) + "}"

# BrandelRun= True
# BrandSourcingRun=True
# ProductSourcingRun=True
# PERun=True
# RevenueResponseRun=True
# SERun=True


<!-- ### PBI extract for WoB and GM -->

In [87]:
if PERun: 
    client_manuf_dax = ", ".join(f'"{x}"' for x in client_manuf)
    conn_str = f"Provider=MSOLAP.8;Data Source={server};Initial Catalog={dataset_name};Timeout=600;"


In [88]:
def execute_dax_query(entity_name, area):
    outputdic = {}
    # Construct filter condition dynamically
    columns = ["Product Names WoB %","Gross Margin %"]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        ADDCOLUMNS(
            SUMMARIZE(
                ProductNames,
                ProductNames[ProductName]
                ),
            {column_exprs}
        ),
            ProductNames[SourceName]= "Consumerresearchname",
            TREATAS(
                {{"{categories[0]}"}} , 
                Products[Category]
            ),
            TREATAS(
                {{{client_manuf_dax}}},
                Products[Top Companies]
            ),
            TREATAS(
                {p12m_dax},
                Calendar[MonthYear]
            ),
            TREATAS(
                {{"{entity_name}"}},
                Market[{area}]
            )
    )  
    """
 
    try:
        with adodbapi.connect(conn_str) as conn:
            with conn.cursor() as cursor:
                cursor.execute(dax_query)
                columns = [desc[0] for desc in cursor.description]
                data = cursor.fetchall()
 
                df = pd.DataFrame(data, columns=columns)
                df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
                df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
                outputdic = df
                print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name}: {e}")
   
    return outputdic
 
# Process data concurrently
def process_dax_queries(entity_hierarchy):
    with ThreadPoolExecutor(max_workers=8) as executor:
       
        dfs_results = {}
        futures = []
        
        for area, entity_list in entity_hierarchy:
            for entity in entity_list:
                future = executor.submit(execute_dax_query, entity, area)
                futures.append((future, entity, area))

        for future, entity, area in futures:
            df = future.result()
            dfs_results[(entity)] = df

        # Save results
        output_path = "Pricing CBC Datasets Test/Pricing_CBC.pkl"
        pd.to_pickle(dfs_results, output_path)
        print(f"All DataFrames saved to {output_path}.")
 

if PERun:
    process_dax_queries(entity_hierarchy) 
    pbi = pd.read_pickle("Pricing CBC Datasets Test/Pricing_CBC.pkl")
    for key, df in pbi.items():
        #if not df.empty:
        df.rename(columns={'Product Names WoB %':'WOB%', 'Gross Margin %':'GM%'}, inplace=True)
        #pbi[key] = df
        #else:
        #    print(f"Warning: DataFrame for {key} is empty.")

Query executed successfully for Walmart.
All DataFrames saved to Pricing CBC Datasets Test/Pricing_CBC.pkl.


<!-- ### Read Data in Excel -->

In [89]:
if BrandelRun:
    sheet_name='Brand elasticity'
    brandElasticity_ori=pd.read_excel(file_path,sheet_name)

In [90]:
def normalize_columns(df):
    column_mapping = {
        'product': 'Product',
        'Product': 'Product',
        'SKU': 'Product',
        # Add more mappings as necessary
    }
    
    # Apply the column mapping
    df.rename(columns=column_mapping, inplace=True)
    
    return df


In [91]:
if BrandSourcingRun:
    plusbranding = pd.read_excel(file_path,"Plus Brand Sourcing").rename(columns={'Unnamed: 0': 'Product'})
    Minusbranding = pd.read_excel(file_path,"Minus Brand Sourcing").rename(columns={'Unnamed: 0': 'Product'})
    plusbrandingfair = pd.read_excel(file_path,"Plus Brand Sourcing Fair share").rename(columns={'Unnamed: 0': 'Product'})
    Minusbrandingfair = pd.read_excel(file_path,"Minus Brand Sourcing Fair share").rename(columns={'Unnamed: 0': 'Product'})
    Minusbrandingfair = Minusbrandingfair[[col for col in Minusbrandingfair.columns if 'Unnamed' not in col]]
    plusbranding = normalize_columns(plusbranding)
    Minusbranding = normalize_columns(Minusbranding)
    plusbrandingfair = normalize_columns(plusbrandingfair)
    Minusbrandingfair = normalize_columns(Minusbrandingfair)

In [92]:
if ProductSourcingRun: 
    plus = pd.read_excel(file_path,"Plus Sourcing")
    Minus = pd.read_excel(file_path,"Minus Sourcing")
    plusfair = pd.read_excel(file_path,"Plus Sourcing Fair share")
    Minusfair = pd.read_excel(file_path,"Minus Sourcing Fair share")
    plus = normalize_columns(plus)
    Minus = normalize_columns(Minus)
    plusfair = normalize_columns(plusfair)
    Minusfair = normalize_columns(Minusfair)

In [93]:
if PERun: 
    pe = pd.read_excel(file_path,"PE")
    group_list = pe['Grouping'].unique().tolist()
    pe = normalize_columns(pe) 

<!-- ### Data Cleaning -->

In [94]:
if PERun:
    from openpyxl import load_workbook
    from openpyxl.utils import get_column_letter

    # Load the workbook and select the sheet
    wb = load_workbook(file_path, data_only=True)
    sheet = wb['PE']
    pe["Share P5 - Base Share"] = pd.Series([float(0)] * len(pe))
    pe["PE P5-P6"] = pd.Series([float(0)] * len(pe))
    # Check for yellow cells and adjust shares
    for row in range(1, len(pe)+2):
        for col in range(1, pe.shape[1]+ 2):
            cell = sheet.cell(row=row, column=col)
            if cell.fill.fgColor.rgb == 'FFFFFF00':
                column_name = sheet.cell(1,col).value 
                col_num=int(column_name[1])
                
                for i in range(5,col_num-1,-1):
                    if i == col_num:
                        pe[f"Share P{i} - Base Share"][row-2] =0
                        pe[f"PE P{i}-P{i+1}"][row-2]=0
                    else:
                        pe[f"Share P{i} - Base Share"][row-2] = pe[f"Share P{i-1} - Base Share"][row-2]
                        pe[f"PE P{i}-P{i+1}"][row-2] =  pe[f"PE P{i-1}-P{i}"][row-2]
                

In [95]:
if PERun:
    # Create the cleaned column
    pbi[marketsub]["ProductNameUpp"] = pbi[marketsub]["ProductName"].str.upper().apply(lambda x: ' '.join(str(x).split()))

    # Also create in pe
    pe["ProductUpp"] = pe["Product"].str.upper().apply(lambda x: ' '.join(str(x).split()))

    # Now merge on these cleaned columns
    pepbijoin = pd.merge(
        pe,
        pbi[marketsub][['ProductNameUpp', 'WOB%', 'GM%']],
        how='left',
        left_on='ProductUpp',
        right_on='ProductNameUpp'
    )

In [96]:
if RevenueResponseRun: 
    from openpyxl import load_workbook

    # Load the workbook and select the specific sheet
    wb = load_workbook(file_path, data_only=True)
    sheet = wb['Revenue Response']

    # Create an empty color mask matrix
    bgcol = np.empty([sheet.max_row, sheet.max_column], dtype=object, order='C')

    # Cycle through all cells to get colors
    for row in range(sheet.max_row):
        for column in range(sheet.max_column):
            cell = sheet.cell(row + 1, column + 1)
            
            # Check if the cell font color is not red (hex 'FFFF0000'), if it is, set the value to empty
            color_in_hex_index = cell.value if cell.font.color.rgb != 'FFFF0000' else ''
            
            # Assign the value to the bgcol array
            bgcol[row, column] = str(color_in_hex_index)

    # Convert the bgcol array to a pandas DataFrame
    colormask = pd.DataFrame(bgcol)

    # Print the shape of the DataFrame
    #print(colormask.shape)

    # Adjust columns names and content
    # Define ranges for slicing the DataFrame
    if len([]) == 0:
        SheetContentRanges = range(0, colormask.shape[0]), range(0, colormask.shape[1])

    # Slice the DataFrame according to defined ranges
    colormask = colormask.iloc[SheetContentRanges]

    # Reset index and update column names
    colormask.reset_index(drop=True, inplace=True)
    colormask.columns = colormask.iloc[0]  # Set the first row as the header
    colormask = colormask.iloc[1:, :]  # Remove the first row (header) and the first column

    # Drop specific columns
    colormask = colormask.drop(columns=['Select SKUs\n(with "x")', 'Possible steps (d/u)', 'Base case share'])

    # Display the final DataFrame
    #print(colormask)


In [97]:
def cleaning_revenue(colormask):
    """
    Clean revenue data from a DataFrame.
 
    Parameters:
    - colormask: The DataFrame containing revenue data.
 
    Returns:
    - dfList: A list of cleaned DataFrames.
    """
    dfList = []  # Initialize an empty list to store cleaned DataFrames
    colormask = colormask.loc[:, ~colormask.columns.duplicated(keep='last')]  # Remove duplicate columns
    for i in range(1, colormask.shape[0] + 1):
        df = colormask.iloc[i - 1:i]  # Extract a single row DataFrame
        df = df.replace("", np.nan)  # Replace empty cells with NaN
        df.dropna(axis=1, inplace=True)  # Drop columns with NaN values
        dfList.append(df)  # Append cleaned DataFrame to the list
    return dfList  # Return the list of cleaned DataFrames


In [98]:
if RevenueResponseRun: dfList = cleaning_revenue(colormask)

In [99]:
if SERun: 
    sheet_name='SE'
    SE=pd.read_excel(file_path,sheet_name)


In [100]:
def SE_clean(data):
    new_rows = []

    for _, row in data.iterrows():
        # First row: s1 and s2 
        new_rows.append({'Product':row['Product'],
        'Base Size': row['S1 total'], 
        'New Size': row['S2 total'],
        'Size Change':row['size S2 - Base size'],
        'Volume Index': row['Volume Index 2'],
        'Value Index': row['Value Index 2'],
        'Gross Profit':row['Gross Profit 2'],
        'Size Elasticity': row['SE 2'], 
        'SCD PE':row['SCD PE 2'], 
        'PCD PE':row['PCD PE 2']})
        # Second row: s1 and s3 
        new_rows.append({'Product':row['Product'],
        'Base Size': row['S1 total'],
        'New Size': row['S3 total'], 
        'Size Change':row['size S3 - Base size'],
        'Volume Index': row['Volume Index 3'],
        'Value Index': row['Value Index 3'],
        'Gross Profit':row['Gross Profit 3'],
        'Size Elasticity': row['SE 3'], 
        'SCD PE':row['SCD PE 3'], 
        'PCD PE':row['PCD PE 3']})
        
    # Convert the list of new rows back into a DataFrame
    new_df = pd.DataFrame(new_rows)
    new_df = new_df.dropna(thresh=new_df.shape[1] - 7).reset_index(drop=True)
    return new_df


In [101]:
if SERun: SE_cleaned = SE_clean(SE)

<!-- ## Duplication Stage -->

In [102]:
index = [0,1 ,2,3,4,5 ]

duplication = [(len(brandElasticity_ori.columns)+3)//6 if BrandelRun else 0,len(plusbranding.keys())-1 if BrandSourcingRun else 0,len(plus.keys())-1 if ProductSourcingRun else 0, len(group_list) if PERun else 0, colormask.shape[0] if RevenueResponseRun else 0, math.ceil(len(SE_cleaned)/20) if SERun else 0]
section_names = ["Brand Elasticity" if BrandelRun else "","Brand Sourcing Analysis" if BrandSourcingRun else 0,"Product Sourcing Analysis" if ProductSourcingRun else 0,"Price Elasticity Curve" if PERun else 0,"Revenue Response Analysis" if RevenueResponseRun else 0,"Size Elasticity" if SERun else 0]

path = os.getcwd() + '//Pricing CBC base Oct 2024.pptx'
new_pre = os.getcwd() + '//slide duplicated.pptx'

print(index)
print(duplication)
print(section_names)
print(len(index))
print(len(duplication))
print(len(section_names))
print(sum(duplication))

[0, 1, 2, 3, 4, 5]
[4, 16, 48, 17, 48, 5]
['Brand Elasticity', 'Brand Sourcing Analysis', 'Product Sourcing Analysis', 'Price Elasticity Curve', 'Revenue Response Analysis', 'Size Elasticity']
6
6
6
138


In [103]:
# slideDuplication(index,duplication,section_names,path,new_pre)

In [104]:
if len(slides_name) >0:
    valid_sections = [(i, s) for i, s in enumerate(section_names) if s is not None]
    indices = [i for i, s in valid_sections if any(sub.lower() == s.lower() for sub in slides_name)]
    filtered_section_names = [section_names[i] for i in indices]
    filtered_duplication = [duplication[i] for i in indices]
    filtered_index = [index[i] for i in indices]
    slideDuplication(filtered_index,filtered_duplication,filtered_section_names,path,new_pre)
else:
    slideDuplication(index,duplication,section_names,path,new_pre)

In [105]:
prs = Presentation(new_pre)

<!-- ### Slide 1 -->

In [106]:
if BrandelRun:
    try:
        if 0 in filtered_index:
            dup_list = filtered_duplication
            run_slide = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        for sli in range(dup_list[0]):
            start=sli*6
            end=start+3
            brandElasticity= brandElasticity_ori[brandElasticity_ori.columns[start:end]]
            brandElasticity=brandElasticity.dropna()
            
            BrandElasticity(prs,brandElasticity,1,categories[0],position=0+sli) 

<!-- ### Slide 2 -->

In [107]:
if BrandSourcingRun:
    try:
        if 1 in filtered_index:
            dup_list = filtered_duplication
            run_slide = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        BrandSourcingAnalysis(prs,plusbranding,Minusbranding,plusbrandingfair,Minusbrandingfair,dup_list[1],position=sum(dup_list[:1]))

<!-- ### Slide 3 -->

In [108]:
if ProductSourcingRun:
    try:
        if 2 in filtered_index:
            dup_list = filtered_duplication
            run_slide = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        ProductSourcingAnalysis(prs,plus,Minus,plusfair,Minusfair,dup_list[2],position=sum(dup_list[:2]))

<!-- ### Slide 4 -->

In [109]:
if PERun:
    try:
        if 3 in filtered_index:
            dup_list = filtered_duplication
            run_slide = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        PriceElasticityCurve(prs, pepbijoin, group_list, dup_list[3],position=sum(dup_list[:3]))

<!-- ### Slide 5 -->

In [110]:
if RevenueResponseRun: 
    try:
        if 4 in filtered_index:
            dup_list = filtered_duplication
            run_slide = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        revenue_response(prs,dfList, dup_list[4],position=sum(dup_list[:4]))

Updated value axis to min: 60.0, max: 130.0
Updated value axis to min: 60.0, max: 140.0
Updated value axis to min: 50.0, max: 130.0
Updated value axis to min: 60.0, max: 150.0
Updated value axis to min: 70.0, max: 150.0
Updated value axis to min: 50.0, max: 130.0
Updated value axis to min: 60.0, max: 150.0
Updated value axis to min: 50.0, max: 160.0
Updated value axis to min: 60.0, max: 140.0
Updated value axis to min: 60.0, max: 130.0
Updated value axis to min: 50.0, max: 140.0
Updated value axis to min: 50.0, max: 140.0
Updated value axis to min: 60.0, max: 130.0
Updated value axis to min: 60.0, max: 130.0
Updated value axis to min: 60.0, max: 130.0
Updated value axis to min: 60.0, max: 140.0
Updated value axis to min: 70.0, max: 160.0
Updated value axis to min: 60.0, max: 160.0
Updated value axis to min: 60.0, max: 170.0
Updated value axis to min: 70.0, max: 160.0
Updated value axis to min: 60.0, max: 130.0
Updated value axis to min: 70.0, max: 140.0
Updated value axis to min: -1800

<!-- ### Slide 6 -->

In [111]:
if SERun: 
    try:
        if 5 in filtered_index:
            dup_list = filtered_duplication
            run_slide = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        SE_Slide(prs,SE_cleaned,dup_list[5],position=sum(dup_list[:5]))

Processing Slide 1
Processing Slide 2
Processing Slide 3
Processing Slide 4
Processing Slide 5


<!-- ### Final -->

In [112]:
from datetime import date
today = str(date.today())
outputPath=os.getcwd() + "\\Pricing CBC "+marketsub+" output "+today+".pptx"
prs.save(outputPath)
app = win32.Dispatch("PowerPoint.Application")
#presentation = app.Presentations.Open(outputPath)

In [113]:
def get_shape_number_partial(shapes, text):
    """
    Get the index of a shape with specific text.

    Parameters:
    - shapes (list): A list of shapes in a slide.
    - text (str): The text to search for in the shapes.

    Returns:
    - int or None: The index of the shape if found, None otherwise.
    """
    for shape_index in range(len(shapes)):
        if shapes[shape_index].has_text_frame:  # Check if the shape has a text frame
            if text in shapes[shape_index].text:  # Compare the shape's text with the given text
                return shape_index  # Return the index of the shape if text matches
    return None  # Return None if the shape with the given text is not found

In [114]:
def getChartNum(shapes):
    """
    Extractcharts from a collection of shapes in a PowerPoint slide.

    Parameters:
    - shapes: A collection of shapes in a PowerPoint slide.

    Returns:
    - shape_index: shape number in selecton pane
    """
    shape_index = []
    for shape_index in range(len(shapes)):
        if shapes[shape_index].has_chart:
            return shape_index

In [115]:
if RevenueResponseRun:  
    sls = [sl for sl in prs.slides][sum(duplication[:4]):sum(duplication[:5])]
    shapes = prs.slides[sum(duplication[:4])].shapes
    subtitleNumber = get_shape_number_partial(shapes, "Revenue Response Curve by Price Point")
    chartNumber = getChartNum(shapes)
    print(chartNumber)

4


In [116]:
if RevenueResponseRun:  
    prod_names=[i.shapes[subtitleNumber].text.split(" | ")[1] for i in sls]
    prod_names=list(set(prod_names))
    len(prod_names)
    filter_dps=pd.read_excel(file_path, sheet_name="PE")
    filter_dps=filter_dps.melt(id_vars=["Product"],value_vars=["P1","P2","P3","P4","P5"])
    filter_dps=filter_dps.groupby("Product")["value"].unique().reset_index()
    filter_dps=dict(zip(filter_dps["Product"],filter_dps["value"]))
    len(filter_dps.keys())
    set(prod_names)==set(filter_dps.keys())
    charts=[s.shapes[chartNumber] for s in sls]
    [float(i[0].replace(currency,"")) for i in charts[0].chart.plots[0].categories.flattened_labels]
    dfs=[]
    for num,chart in enumerate(charts):
        temp=pd.DataFrame()
        for i in chart.chart.series:
            temp[i.name]=i.values
        temp["x_axis"]=[float(j[0].replace(currency,"")) for j in chart.chart.plots[0].categories.flattened_labels]
        prod_name=sls[num].shapes[subtitleNumber].text.split(" | ")[1]
        temp=temp[(temp["x_axis"].isin(filter_dps[prod_name]))|(temp["Volume Ix"]==100)]
        dfs.append(temp)
    len(dfs)       

    org_dfs=[]
    for num,chart in enumerate(charts):
        temp=pd.DataFrame()
        for i in chart.chart.series:
            temp[i.name]=i.values
        temp["x_axis"]=[float(j[0].replace(currency,"")) for j in chart.chart.plots[0].categories.flattened_labels]
        prod_name=sls[num].shapes[subtitleNumber].text.split(" | ")[1]
        org_dfs.append(temp)
    len(org_dfs)       

In [117]:
if RevenueResponseRun:  
  final_res=[]
  dfs=org_dfs
  for num_index,df in enumerate(dfs) :
    df["ZeroPoint"]=np.where((df["Volume Ix"]==100),-1,np.nan)
    df["ZeroPoint"]=df["ZeroPoint"].bfill()
    df["ZeroPoint"]=df["ZeroPoint"].fillna(1)
  

    df["Value Ix delta"]=df["Value Ix"].diff() 
    df["Gross Profit Ix delta"]=df["Gross Profit Ix"].diff()

    df["Value Ix delta"]=df["Value Ix delta"]*df["ZeroPoint"]
    df["Gross Profit Ix delta"]=df["Gross Profit Ix delta"]*df["ZeroPoint"]



    df["color"]=np.select([(df["Value Ix delta"]>0)&(df["Gross Profit Ix delta"]>0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]>0)&(df["Gross Profit Ix delta"]<0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]>0)&(df["Gross Profit Ix delta"]==0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]<0)&(df["Gross Profit Ix delta"]>0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]<0)&(df["Gross Profit Ix delta"]<0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]<0)&(df["Gross Profit Ix delta"]==0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]==0)&(df["Gross Profit Ix delta"]>0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]==0)&(df["Gross Profit Ix delta"]<0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]==0)&(df["Gross Profit Ix delta"]==0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]>0)&(df["Gross Profit Ix delta"]>0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]>0)&(df["Gross Profit Ix delta"]<0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]>0)&(df["Gross Profit Ix delta"]==0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]<0)&(df["Gross Profit Ix delta"]>0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]<0)&(df["Gross Profit Ix delta"]<0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]<0)&(df["Gross Profit Ix delta"]==0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]==0)&(df["Gross Profit Ix delta"]>0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]==0)&(df["Gross Profit Ix delta"]<0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]==0)&(df["Gross Profit Ix delta"]==0)&(df["Value Ix"]<=100)],
                          [
  "Green|Profit & Revenue Increase",
  "Yellow|Revenue Increase & Profit Dilution",
  "Yellow|Revenue Increase & Flat Profit",
  "Yellow|Revenue Dilution & Profit Increase",
  "Red|Revenue & Profit Dilution",
  "Red|Revenue Dilution & Flat Profit",
  "Yellow|Flat Revenue & Profit Increase",
  "Red|Flat Revenue & Profit Dilution",
  "Yellow|Flat Revenue & Profit",
  "Yellow|Profit & Revenue Increase",
  "Yellow|Revenue Increase & Profit Dilution",
  "Yellow|Revenue Increase & Flat Profit",
  "Yellow|Revenue Dilution & Profit Increase",
  "Red|Revenue & Profit Dilution",
  "Red|Revenue Dilution & Flat Profit",
  "Yellow|Flat Revenue & Profit Increase",
  "Red|Flat Revenue & Profit Dilution",
  "Yellow|Flat Revenue & Profit",
  ],"")
    

    df=org_dfs[num_index].merge(df,how="left")

    df=df.reset_index(drop=True)
    df=df.iloc[1:,:]
    df["color"]=df["color"].bfill()
    df["grouping_same_color"]=DetermineShapeNumber(df["color"].apply(lambda x : x.split("|")[0]).to_list())
    groups=df.groupby("grouping_same_color")["color"].apply(lambda x : pd.Series(x).mode()[0]).reset_index()
    df=df.drop(columns=["color"]).merge(groups)
    
    df["shape_number"]=DetermineShapeNumber(df["color"].to_list())
    df["color_num"]=df["color"]+"_"+df["shape_number"].astype(str)
    dist=df["color_num"].to_list()
    full_lenght=len(dist)
    dist_list=[[0,0,dist[0] ]]
    current=dist[0]
    for i in range(len(dist)):
      if current == dist[i]:
        dist_list[-1][1]+=1
      else :
        dist_list.append([i,i+1,dist[i]])
        current=dist[i]
    res=[(int(round(i[0]/len(dist),2)*100),int(round(i[1]/len(dist),2)*100),i[2].split("_")[0]) for i in dist_list ]  
    final_res.append(res)
    

In [118]:
if RevenueResponseRun:  
    full_width=Cm(15.7)
    left_start=Cm(2.7)
    for num,slide in enumerate(sls):
        for sh in final_res[num]:
            added_shape=AddRectangle(slide,left_start + (sh[0]/100) * full_width, ((sh[1]-sh[0])/100) * full_width,sh[2] )
        


In [119]:
if RevenueResponseRun:  
    prs.save(outputPath)

In [120]:
new_order = []
current_max = len(new_order)
group = pepbijoin['Product'][pepbijoin['Grouping'] == group_list[0]]


In [121]:
app = win32.Dispatch("PowerPoint.Application")
presentation = app.Presentations.Open(outputPath)

In [122]:
# app = win32.Dispatch("PowerPoint.Application")
# presentation = app.Presentations.Open(outputPath)

new_order = []
current_max = len(new_order)

if BrandelRun: 
    current_max += duplication[0]
    new_order.extend(range(len(new_order) + 1, current_max + 1))
if BrandSourcingRun: 
    current_max += duplication[1]
    new_order.extend(range(len(new_order) + 1, current_max + 1))
if SERun: 
    total = sum(duplication)
    count = duplication[5]
    new_order.extend(range(total - count + 1, total + 1))

#Reordering PE, RR, Sourcing
if PERun:
    startPE = sum([duplication[0],duplication[1],duplication[2]])
    startSou= sum([duplication[0],duplication[1]])
    startRR= sum([duplication[0],duplication[1],duplication[2],duplication[3]])
    for group_num,group_name in enumerate(group_list):
        new_order.append(startPE+group_num+1)
        group = pepbijoin['Product'][pepbijoin['Grouping'] == group_name]
        for i, name in enumerate(group):
            row_numbers = []
            for i, df in enumerate(dfList):
                if (df['Product'] == name).any():
                    row_numbers.append(i) 
            if RevenueResponseRun: new_order.append(startRR + row_numbers[0] + 1)
            if ProductSourcingRun: new_order.append(startSou + row_numbers[0] + 1)
if PERun== False:
    startSou=sum([duplication[0],duplication[1]])
    startRR= sum([duplication[0],duplication[1],duplication[2],duplication[3]])
    if RevenueResponseRun: new_order.extend(range(startRR, duplication[4]))
    if ProductSourcingRun: new_order.extend(range(startRR, duplication[2]))


print(f"New slide order (1-based): {new_order}")

def reorder_slides_in_place(presentation, new_order):
    total_slides = presentation.Slides.Count
    
    slides_temp = []
    
    for slide_index in new_order:
        if slide_index <= total_slides:
            original_slide = presentation.Slides(slide_index)
            original_slide.Copy()
            time.sleep(0.5) 
            new_slide = presentation.Slides.Paste(presentation.Slides.Count + 1)
            slides_temp.append(new_slide)
    
    for i in range(total_slides, 0, -1):
        presentation.Slides(i).Delete()
    
    print(f"Reordered {len(slides_temp)} slides successfully")
    return presentation

# Reorder the slides
presentation = reorder_slides_in_place(presentation, new_order)

reordered_outputPath = os.getcwd() + "\\Pricing CBC "+marketsub+" reordered output "+today+".pptx"
def remove_all_sections(presentation):
    try:
        section_count = presentation.SectionProperties.Count
        print(f"Removing {section_count} sections...")
        for i in range(section_count, 0, -1):
            presentation.SectionProperties.Delete(i, False)  # False = keep slides
        print("All sections removed.")
        return presentation
    except Exception as e:
        print(f"Error removing sections: {e}")

presentation = remove_all_sections(presentation)

# Add Section names
presentation.SectionProperties.AddBeforeSlide(1,section_names[0])
presentation.SectionProperties.AddBeforeSlide(duplication[0]+1,section_names[1] )
presentation.SectionProperties.AddBeforeSlide(duplication[0]+duplication[1]+1,section_names[5] )
presentation.SectionProperties.AddBeforeSlide(duplication[0]+duplication[1]+duplication[5]+1,"PE, Revenue Responses, Sourcing by product")


presentation.SaveAs(reordered_outputPath)

print(f"Reordered presentation saved as: {reordered_outputPath}")
print("Slide reordering complete! All formatting preserved.")

New slide order (1-based): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 134, 135, 136, 137, 138, 69, 86, 21, 91, 26, 70, 87, 22, 88, 23, 89, 24, 90, 25, 94, 29, 95, 30, 96, 31, 71, 92, 27, 93, 28, 72, 97, 32, 101, 36, 73, 98, 33, 99, 34, 100, 35, 106, 41, 74, 102, 37, 103, 38, 104, 39, 105, 40, 107, 42, 75, 108, 43, 109, 44, 76, 110, 45, 112, 47, 114, 49, 77, 111, 46, 113, 48, 78, 115, 50, 116, 51, 79, 117, 52, 119, 54, 80, 118, 53, 81, 120, 55, 124, 59, 125, 60, 82, 121, 56, 123, 58, 128, 63, 129, 64, 83, 122, 57, 126, 61, 127, 62, 84, 130, 65, 85, 131, 66, 132, 67, 133, 68]
Reordered 138 slides successfully
Removing 6 sections...
All sections removed.
Reordered presentation saved as: c:\Users\Ali Salem\Desktop\App_Update\Pricing CBC\Pricing CBC Walmart reordered output 2025-08-31.pptx
Slide reordering complete! All formatting preserved.
